<a href="https://colab.research.google.com/github/askolesov0608/LAB-2/blob/master/003Lab3_Ex1_1_cifar_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from keras.utils import get_file
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
from random import seed, randint
import time
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Указываем путь сохранения лога на Google Диске
log_dir = '/content/drive/My Drive/Colab Notebooks/Logs'
os.makedirs(log_dir, exist_ok=True)

# Настройка CSVLogger для сохранения на Google Диске
drive_log_path = os.path.join(log_dir, 'drive_log.csv')
drive_csv_logger = CSVLogger(drive_log_path, append=True, separator=';')

In [ ]:
print(get_file('dummy', origin='https://example.com'))

In [ ]:
# reading dataset

(train_X, train_Y), (val_X, val_Y) = cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# shapes of data structures
print('tamaños conjunto de entrenamiento X:', train_X.shape, '  Y:', train_Y.shape)
print('tamaños conjunto de validación X:', val_X.shape, '  Y:', val_Y.shape )
print("Размер обучающего набора изображений: ", x_train.shape)
print("Размер обучающего набора меток: ", y_train.shape)
print("Размер тестового набора изображений: ", x_test.shape)
print("Размер тестового набора меток: ", y_test.shape)


In [ ]:
# displaying randomly chosen images
seed(1)
plt.figure(1, figsize=(10,6))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.tight_layout()
    j = randint(0, train_Y.shape[0]-1)
    plt.imshow(train_X[j], interpolation='lanczos')
    plt.title('class:{0}, pos:{1}'.format(train_Y[j], j))
    plt.axis("off")
plt.show()

In [ ]:
# preprocessing data

num_classes = 10
train_X = train_X.astype('float32')
val_X = val_X.astype('float32')

train_X /= 255
val_X /= 255

train_Y = keras.utils.to_categorical(train_Y, num_classes)
val_Y = keras.utils.to_categorical(val_Y, num_classes)

print('tamaños conjunto de entrenamiento X:', train_X.shape, '  Y:', train_Y.shape)
print('tamaños conjunto de validación X:', val_X.shape, '  Y:', val_Y.shape )

In [ ]:
# Нормализация изображений
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Преобразование меток в категориальный формат
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(y_train)
print(y_test)


In [ ]:
# model definition
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=train_X.shape[1:])) # input_shape=(32,32,3)
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# hyperparameters
opt = SGD(learning_rate=0.01, momentum=0.9)
# opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

batch_size = 100
epochs = 50

best_model_path = 'best_model'
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath = best_model_path,
    save_weights_only = True,
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    save_freq='epoch'
)

log_path='log.csv'
history_logger = keras.callbacks.CSVLogger(
    filename=log_path,
    separator=",",
    append=True
)

# Optimización del modelo (entrenamiento)
start_time = time.time()
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True, callbacks=[model_checkpoint, history_logger])
print('Tiempo: {0:.2f} min'.format((time.time()-start_time)/60))




In [ ]:
# recovering the best model
model.load_weights(best_model_path)

In [ ]:
# testing de model
train_res = model.evaluate(train_X, train_Y, verbose=0)
val_res = model.evaluate(val_X, val_Y, verbose=0)

print('Resultados ENTRENAMIENTO: loss={:.4f}, accuracy={:.4f}'.format(train_res[0], train_res[1]))
print('Resultados VALIDACIÓN: loss={:.4f}, accuracy={:.4f}'.format(val_res[0], val_res[1]))

# plotting learnin curves
plt.figure(2)
plt.plot(range(1,epochs+1), history.history['accuracy'])
plt.plot(range(1,epochs+1), history.history['val_accuracy'])
plt.xticks(range(1,epochs+1))
plt.xlim(1,epochs)
plt.ylim(0, 1)
plt.show()